In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from pathlib import Path

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/samira/Desktop/Testing_LLM/.conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
import getpass
import os
os.environ["GVO78KFjnCgLq4jv7jvRd6SnuDTXbUOr03l4rcUo"] = getpass.getpass()

In [3]:
from langchain_cohere import ChatCohere

# Directly pass the API key to the ChatCohere class
cohere_api_key = "GVO78KFjnCgLq4jv7jvRd6SnuDTXbUOr03l4rcUo"  # Replace with your actual API key
llm = ChatCohere(model="command-r", cohere_api_key=cohere_api_key)

In [4]:
# SPECIFY THE FOLDER PATH WHERE ALL YOUR .TXT FILES ARE LOCATED!!
#folder_path = "documents"  # Replace with your folder path

# Use pathlib to list all .txt files in the folder
#file_paths = list(Path(folder_path).rglob("*.txt"))

# Load documents from each .txt file in the folder
#all_documents = []
#for file_path in file_paths:
#    loader = TextLoader(str(file_path), encoding='utf-8')  # Convert Path object to string
#    documents = loader.load()
#    all_documents.extend(documents)  # Add documents to the list

#from langchain_community.document_loaders import PyPDFLoader #for PDFs
#loader = PyPDFLoader("documents/Liquid_NN.pdf")
#documents_pdf = loader.load()

In [5]:
#from langchain_community.document_loaders import PyPDFLoader #for PDFs
#loader = PyPDFLoader("documents/Liquid_NN.pdf")
#documents_pdf = loader.load()

In [4]:
from langchain.document_loaders import TextLoader #for .txt files

# Load the local .txt file (replace with the path to your text file)
loader = TextLoader("documents/A thousand ways to make money.txt", encoding='utf-8')

# Load the documents
documents_txt = loader.load()

# Print the content of the loaded documents
for doc in documents_txt:
    print(doc.page_content)

The Project Gutenberg eBook of One Thousand Ways to Make Money
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: One Thousand Ways to Make Money

Author: Page Fox

Release date: November 19, 2017 [eBook #56006]

Language: English

Credits: Produced by Chuck Greif, MFR and the Online Distributed
        Proofreading Team at http://www.pgdp.net (This file was
        produced from images generously made available by The
        Internet Archive)


*** START OF THE PROJECT GUTENBERG EBOOK ONE THOUSAND WAYS TO MAKE MONEY ***




Produced by Chuck Greif, MFR and the Online Distributed
Proofre

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents_txt)

In [6]:
%pip install -U langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

In [8]:
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

/Users/samira/Desktop/Testing_LLM/.conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/samira/Desktop/Testing_LLM/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [10]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [11]:
import chromadb
chroma_client = chromadb.Client()

In [12]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [13]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

/Users/samira/Desktop/Testing_LLM/.conda/lib/python3.11/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [15]:
rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

In [18]:
#rag_chain.invoke("How should I start in business?")

In [16]:
%pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip uninstall streamlit -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: streamlit 1.38.0
Uninstalling streamlit-1.38.0:
  Successfully uninstalled streamlit-1.38.0
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 35.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
#system_prompt = """
#You are an immigration expert for Austria. Answer the user's questions about immigration policies, processes, and requirements with accurate and helpful information.
#"""

# Define the function to run the RAG pipeline with the system prompt
#def run_rag_pipeline_with_system_prompt(question):
    # Combine system prompt, retrieved documents, and user question
#    system_context = system_prompt + "\n\n"
    
#    # Combine the system prompt and the user's question
#    formatted_question = system_context + f"Question: {question}"
    
#    # Invoke the RAG pipeline with the formatted input
#    return rag_chain.invoke(formatted_question)

In [20]:
import streamlit as st# Define the function to run the RAG pipeline
def run_rag_pipeline(question):
    return rag_chain.invoke(question)

# Streamlit interface
st.title("Immigration Bot")

# User input field
user_input = st.text_input("Hello there. I will assist you with questions regarding your immigration to Austria.")

# If the user submits a question, run the RAG pipeline and display the response
if user_input:
    response = run_rag_pipeline(user_input)
    st.write(f"Chatbot: {response}")

2024-10-11 12:13:05.642 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.777 
  command:

    streamlit run /Users/samira/Desktop/Testing_LLM/.conda/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-11 12:13:05.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-11 12:13:05.